Creation 20/4/18

This notebook creates a table which contains the categorised information of all people in a household of a tenancy on a given arrears event data (start of an arrears streak). The table is export to a csv at the end of the code.

Input reqiured:
- User credentials to database to get TT_PARTIES_VIEW table
- Import view created from SQL command: SQL_Create_view_link_tcyrefno_to_all_people_in_household.sql
- Arrears events tables for weekly and monthly accounts: Arrears_Events_Monthly_aug.csv; Arrears_Events_Weekly_aug.csv

Output:
- Arrears_Events_PersonsInfo.csv. This csv contains all the HOP info on a arrears event start date


Problems encountered:
- Missing tenancy refs on an arrears start date (not clear if this is due to incorrect dates or tenancies in the ArrearsEvents tables and TT_PARTIES_VIEW not matching


# Import modules

In [2]:
%matplotlib inline
import pymssql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
from collections import defaultdict, Counter
import math

In [2]:
## PLEASE ADD YOUR USER CREDENTIALS TO CONNECT TO DATABASE IN THIS CELL

DB_SERVER = '51.145.25.129'
USER_NAME = 'notting_hill_team'
PASSWORD = '4-f+7-XVP.^FX<<_'
DATABASE_NAME = "PIVIGO"

conn = pymssql.connect(DB_SERVER, USER_NAME ,PASSWORD , DATABASE_NAME)

## Read in TT_PARTIES_VIEW
This contains the info for all persons living in a tenancy household

In [7]:
sql_com = ("SELECT * FROM [TT_PARTIES_VIEW]").encode("utf-8")
TT_Parties_df = pd.read_sql_query(sql_com, conn)

In [8]:
# Look at dataframe created
TT_Parties_df.head(10)

,tcy_refno,hou_refno,tin_main_tenant_ind,move_in_date,move_out_date,Person Reference,DATE OF BIRTH,Language (written),Gender,Nationality,DISABLED,OAP,Marital Status,Ethnic Origin,AT RISK INDICATOR,Sexual Orientation
0,1.0,527.0,Y,1996-09-02,1997-05-04,22356.0,1947-11-13,ENGLISH,M,BRITISH,N,N,COH,1,N,HET
1,2.0,7459.0,Y,1989-05-29,NaT,29288.0,1963-08-05,ENGLISH,F,BRITISH,N,N,None,12,N,None
2,2.0,7459.0,None,1990-05-24,2010-05-16,41191.0,1990-05-24,ENGLISH,M,BRITISH,N,N,SING,4,N,None
3,2.0,7459.0,None,1989-05-29,2012-09-30,41192.0,1980-07-31,None,M,None,N,N,None,None,N,None
4,3.0,7768.0,Y,1990-07-16,NaT,29597.0,1941-12-21,ENGLISH,M,BRITISH,N,N,None,7,Y,None
5,4.0,7694.0,Y,1995-10-09,2003-09-21,29523.0,1974-03-10,None,F,None,N,N,None,7,N,None
6,5.0,7400.0,Y,1986-02-17,NaT,29229.0,1965-08-10,None,M,None,N,N,None,None,N,None
7,6.0,7401.0,Y,1986-02-17,2016-04-03,29230.0,1955-08-25,ENGLISH,M,None,N,N,None,None,N,None
8,7.0,7399.0,Y,1986-02-17,2008-03-30,29228.0,1966-10-13,ENGLISH,M,None,N,N,None,13,N,None
9,9.0,383.0,Y,1995-10-09,NaT,22212.0,1963-01-20,None,F,None,N,N,None,12,N,None


## Partial categorisation of person information
Bin categories in each column into a small number

In [9]:
## Disablity or 'At risk' columns

TT_Parties_catg_df = TT_Parties_df
#create a new column 'disability' that is 1 if either DISABLED or At Risk Indicator are equal to 'Y'
TT_Parties_catg_df['disability']=0 

TT_Parties_catg_df.loc[(TT_Parties_catg_df['DISABLED']=='Y')|(TT_Parties_catg_df['AT RISK INDICATOR']=='Y'), 'disability'] = 'Y'
TT_Parties_catg_df.loc[(TT_Parties_catg_df['DISABLED']=='N') & (TT_Parties_catg_df['AT RISK INDICATOR']=='N'), 'disability'] = 'N'
TT_Parties_catg_df.loc[(TT_Parties_catg_df['DISABLED'].isnull())|(TT_Parties_catg_df['AT RISK INDICATOR'].isnull()), 'disability'] = 'ND'

disability_dumis = pd.get_dummies(TT_Parties_catg_df['disability'], prefix='disability', drop_first=True)
disability_dumis[:5]

TT_Parties_catg_df = pd.concat([TT_Parties_catg_df, disability_dumis], axis=1)


In [10]:
## Add all other categories
## Age will be added from DOB further below

TT_Parties_catg_df = TT_Parties_df

# Binning written language into english,  non-english and null
TT_Parties_catg_df.loc[(TT_Parties_catg_df['Language (written)']!='ENGLISH') & (TT_Parties_catg_df['Language (written)'].notna()), 'Language (written)']='not_ENGLISH'
Lang_writ_dumis2 = pd.get_dummies(TT_Parties_catg_df['Language (written)'], prefix='Lang_writ',dummy_na = True)
TT_Parties_catg_df = pd.concat([TT_Parties_catg_df, Lang_writ_dumis2], axis=1)

# Categorisation of gender
Gender_dumis = pd.get_dummies(TT_Parties_catg_df['Gender'], prefix='Gender', drop_first=True, dummy_na = True)
TT_Parties_catg_df = pd.concat([TT_Parties_catg_df, Gender_dumis], axis=1)

# Binning Nationality into Brit, non-Brit or null
TT_Parties_catg_df.loc[(TT_Parties_catg_df['Nationality']!='BRITISH') & (TT_Parties_catg_df['Nationality'].notna()), 'Nationality']='not_BRITISH'
Nationality_dumis2 = pd.get_dummies(TT_Parties_catg_df['Nationality'], prefix='Nationality',dummy_na = True)
TT_Parties_catg_df = pd.concat([TT_Parties_catg_df, Nationality_dumis2], axis=1)

# Categorisation of Disabled and At Risk into one column called disability
TT_Parties_catg_df['disability']=0 #create a new column 'disability' that is 1 if either DISABLED or At Risk Indicator are equal to 'Y'

TT_Parties_catg_df.loc[(TT_Parties_catg_df['DISABLED']=='Y')|(TT_Parties_catg_df['AT RISK INDICATOR']=='Y'), 'disability'] = 'Y'
TT_Parties_catg_df.loc[(TT_Parties_catg_df['DISABLED']=='N') & (TT_Parties_catg_df['AT RISK INDICATOR']=='N'), 'disability'] = 'N'
TT_Parties_catg_df.loc[(TT_Parties_catg_df['DISABLED'].isnull())|(TT_Parties_catg_df['AT RISK INDICATOR'].isnull()), 'disability'] = 'ND'

disability_dumis = pd.get_dummies(TT_Parties_catg_df['disability'], prefix='disability', drop_first=True)
disability_dumis[:5]
TT_Parties_catg_df = pd.concat([TT_Parties_catg_df, disability_dumis], axis=1)


# OAP will be added later based on age
# Categorisation of OAP 
#OAP_dumis = pd.get_dummies(TT_Parties_catg_df['OAP'], prefix='OAP', drop_first=True, dummy_na = True)
#TT_Parties_catg_df = pd.concat([TT_Parties_catg_df, OAP_dumis], axis=1)

# Binning Marital Status into married, single, other or null
TT_Parties_catg_df.loc[(TT_Parties_catg_df['Marital Status']!='SING') & (TT_Parties_catg_df['Marital Status']!='MAR') & (TT_Parties_catg_df['Marital Status'].notna()), 'Marital Status']='not_married_not_single'
Mar_stat_dumis2 = pd.get_dummies(TT_Parties_catg_df['Marital Status'], prefix='Marital_stat',dummy_na = True)
TT_Parties_catg_df = pd.concat([TT_Parties_catg_df, Mar_stat_dumis2], axis=1)

# Binning Ethncity into white British, black Brit Africa, blac Brit Caribbean, other or null
TT_Parties_catg_df.loc[(~TT_Parties_catg_df['Ethnic Origin'].isin(['1','13','12']) ) & (TT_Parties_catg_df['Ethnic Origin'].notna()), 'Ethnic Origin']='other'
Ethnicity_dumis2 = pd.get_dummies(TT_Parties_catg_df['Ethnic Origin'], prefix='Ethnicity',dummy_na = True)
TT_Parties_catg_df = pd.concat([TT_Parties_catg_df, Ethnicity_dumis2], axis=1)

# Categorisation of AT Risk Factor
At_risk_dumis = pd.get_dummies(TT_Parties_catg_df['AT RISK INDICATOR'], prefix='At_risk', drop_first=True,dummy_na = True)
TT_Parties_catg_df = pd.concat([TT_Parties_catg_df, At_risk_dumis], axis=1)

# Binning into heterosexual, other or null
TT_Parties_catg_df.loc[(TT_Parties_catg_df['Sexual Orientation']!='HET') & (TT_Parties_catg_df['Marital Status'].notna()), 'Sexual Orientation']='not_HET'
Mar_stat_dumis2 = pd.get_dummies(TT_Parties_catg_df['Sexual Orientation'], prefix='Sexual_orientation',dummy_na = True)
TT_Parties_catg_df = pd.concat([TT_Parties_catg_df, Mar_stat_dumis2], axis=1)

## Delete original columns after categorisation
TT_Parties_catg_df = TT_Parties_catg_df.drop(['Language (written)','Gender','Nationality','DISABLED','OAP','Marital Status',
            'Ethnic Origin','AT RISK INDICATOR','Sexual Orientation','disability'],axis=1)

TT_Parties_catg_df.shape

(116900, 40)

In [11]:
TT_Parties_catg_df[0:5]

,tcy_refno,hou_refno,tin_main_tenant_ind,move_in_date,move_out_date,Person Reference,DATE OF BIRTH,Lang_writ_ENGLISH,Lang_writ_not_ENGLISH,Lang_writ_nan,Gender_M,Gender_R,Gender_nan,Nationality_BRITISH,Nationality_not_BRITISH,Nationality_nan,disability_ND,disability_Y,Marital_stat_MAR,Marital_stat_SING,Marital_stat_not_married_not_single,Marital_stat_nan,Ethnicity_1,Ethnicity_12,Ethnicity_13,Ethnicity_other,Ethnicity_nan,At_risk_Y,At_risk_nan,Sexual_orientation_BI,Sexual_orientation_F,Sexual_orientation_GAY,Sexual_orientation_GL,Sexual_orientation_HET,Sexual_orientation_LES,Sexual_orientation_OTH,Sexual_orientation_REFD,Sexual_orientation_TRAN,Sexual_orientation_not_HET,Sexual_orientation_nan
0,1.0,527.0,Y,1996-09-02,1997-05-04,22356.0,1947-11-13,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,2.0,7459.0,Y,1989-05-29,NaT,29288.0,1963-08-05,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2.0,7459.0,None,1990-05-24,2010-05-16,41191.0,1990-05-24,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,2.0,7459.0,None,1989-05-29,2012-09-30,41192.0,1980-07-31,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
4,3.0,7768.0,Y,1990-07-16,NaT,29597.0,1941-12-21,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1


# Read in arrears event tables for weekly and monthly accounts

In [5]:
# importing the arrears events weekly and monthly tables

Arrears_Events_Monthly = pd.read_csv('..\Arrears_Features\Arrears_Events_Monthly_aug.csv')
Arrears_Events_Weekly = pd.read_csv('..\Arrears_Features\Arrears_Events_Weekly_aug.csv')

In [6]:
print(Arrears_Events_Monthly.shape)
Arrears_Events_Monthly.head()

(28108, 9)


,Unnamed: 0,ACCOUNT NUMBER,Tenancy Ref,Current Tenancy,Unnamed: 0.1,ConsecMonths,DatesArrears,LongtermArrearsNum,MonthsInArrears
0,0,9022637,23372,1,0,4,2011-01-31,0,0
1,1,9022637,23372,1,1,20,2011-10-31,0,4
2,2,9022637,23372,1,2,12,2016-08-31,1,24
3,3,9022646,23381,1,3,1,2011-01-31,0,0
4,4,9022646,23381,1,4,22,2011-04-30,0,1


In [18]:
print(Arrears_Events_Weekly.shape)
Arrears_Events_Weekly.head()

(63740, 8)


,Unnamed: 0,ACCOUNT NUMBER,Tenancy Ref,Current Tenancy,DatesArrears,ConsecMonths,LongtermArrearsNum,MonthsInArrears
0,0,9000001,742,1,2011-01-31,2,0,0
1,1,9000001,742,1,2011-09-30,1,0,2
2,2,9000001,742,1,2012-06-30,27,0,3
3,3,9000001,742,1,2014-11-30,40,1,30
4,4,9000006,890,0,2011-01-31,2,0,0


### Add indicator for long term arrears events

In [15]:
Arrears_Events_Monthly['Tenancy Type']  = 'monthly'
Arrears_Events_Monthly['LongTermIndicator']=0 #1 means greater than or equal 6 months
Arrears_Events_Monthly.loc[(Arrears_Events_Monthly['ConsecMonths']>=6), 'LongTermIndicator'] = 1
Arrears_Events_Monthly.head()

,Unnamed: 0,ACCOUNT NUMBER,Tenancy Ref,Current Tenancy,ConsecMonths,DatesArrears,LongtermArrearsNum,MonthsInArrears,Tenancy Type,LongTermIndicator
0,0,9022637,23372,1,4,2011-01-31,0,0,monthly,0
1,1,9022637,23372,1,20,2011-10-31,0,4,monthly,1
2,2,9022637,23372,1,12,2016-08-31,1,24,monthly,1
3,3,9022646,23381,1,1,2011-01-31,0,0,monthly,0
4,4,9022646,23381,1,22,2011-04-30,0,1,monthly,1


In [16]:
Arrears_Events_Weekly['Tenancy Type']  = 'weekly'
Arrears_Events_Weekly['LongTermIndicator']=0 #1 means greater than or equal 6 months
Arrears_Events_Weekly.loc[(Arrears_Events_Weekly['ConsecMonths']>=6), 'LongTermIndicator'] = 1
Arrears_Events_Weekly.head()

,Unnamed: 0,ACCOUNT NUMBER,Tenancy Ref,Current Tenancy,DatesArrears,ConsecMonths,LongtermArrearsNum,MonthsInArrears,Tenancy Type,LongTermIndicator
0,0,9000001,742,1,2011-01-31,2,0,0,weekly,0
1,1,9000001,742,1,2011-09-30,1,0,2,weekly,0
2,2,9000001,742,1,2012-06-30,27,0,3,weekly,1
3,3,9000001,742,1,2014-11-30,40,1,30,weekly,1
4,4,9000006,890,0,2011-01-31,2,0,0,weekly,0


## merge both monthly and weekly into one dataframe


In [17]:
# merge both monthly and weekly into one dataframe

Arrears_Events =pd.concat([Arrears_Events_Monthly,Arrears_Events_Weekly], ignore_index=True)
Arrears_Events.drop(['Unnamed: 0'],axis=1, inplace=True)
Arrears_Events.head()

,ACCOUNT NUMBER,ConsecMonths,Current Tenancy,DatesArrears,LongTermIndicator,LongtermArrearsNum,MonthsInArrears,Tenancy Ref,Tenancy Type
0,9022637,4,1,2011-01-31,0,0,0,23372,monthly
1,9022637,20,1,2011-10-31,1,0,4,23372,monthly
2,9022637,12,1,2016-08-31,1,1,24,23372,monthly
3,9022646,1,1,2011-01-31,0,0,0,23381,monthly
4,9022646,22,1,2011-04-30,1,0,1,23381,monthly


# Build table for aggregated features of all persons in household on start date of arrears event

In [18]:
# Add empty columns to Arrears_Events for features to be added

FeatsColList = ['num_children','num_adults','num_OAPs','avg_adult_age','main_ten_age']
FeatsColList.extend(list(TT_Parties_catg_df.columns.values)[7:])
Arrears_Events = Arrears_Events.reindex(columns=[*Arrears_Events.columns.tolist(), *FeatsColList], fill_value=None)
Arrears_Events.head()


,ACCOUNT NUMBER,ConsecMonths,Current Tenancy,DatesArrears,LongTermIndicator,LongtermArrearsNum,MonthsInArrears,Tenancy Ref,Tenancy Type,num_children,num_adults,num_OAPs,avg_adult_age,main_ten_age,Lang_writ_ENGLISH,Lang_writ_not_ENGLISH,Lang_writ_nan,Gender_M,Gender_R,Gender_nan,Nationality_BRITISH,Nationality_not_BRITISH,Nationality_nan,disability_ND,disability_Y,Marital_stat_MAR,Marital_stat_SING,Marital_stat_not_married_not_single,Marital_stat_nan,Ethnicity_1,Ethnicity_12,Ethnicity_13,Ethnicity_other,Ethnicity_nan,At_risk_Y,At_risk_nan,Sexual_orientation_BI,Sexual_orientation_F,Sexual_orientation_GAY,Sexual_orientation_GL,Sexual_orientation_HET,Sexual_orientation_LES,Sexual_orientation_OTH,Sexual_orientation_REFD,Sexual_orientation_TRAN,Sexual_orientation_not_HET,Sexual_orientation_nan
0,9022637,4,1,2011-01-31,0,0,0,23372,monthly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9022637,20,1,2011-10-31,1,0,4,23372,monthly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9022637,12,1,2016-08-31,1,1,24,23372,monthly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9022646,1,1,2011-01-31,0,0,0,23381,monthly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9022646,22,1,2011-04-30,1,0,1,23381,monthly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
## Define age boundary for OAP
OAP_age_boundary = 65.0


### Loop to calculate person features of tenancy on arrears case date
(N.B. - this might take some time to run)


In [21]:
# Iterate over Arrears_Events and find mathcing TR and date


# Make empty DF for arrears events which have no tenants in at time
Arrears_empty_tenancies = pd.DataFrame(columns=[*Arrears_Events.columns[:7].tolist(),'num_people'])

today = pd.Timestamp.today()
TT_Parties_catg_df = TT_Parties_catg_df.fillna(value={'move_out_date':today})
n_empties = 0 

for i, row_ArrsEvnts in Arrears_Events.iterrows():    # Iterate through arrears cases
    find_TR, find_date = row_ArrsEvnts['Tenancy Ref'], pd.to_datetime(row_ArrsEvnts['DatesArrears'])

    TR_loc_df = TT_Parties_catg_df.loc[TT_Parties_catg_df['tcy_refno']==find_TR]
    
    # Find all people in tenancy at given date
    TRandDate_loc_df = TR_loc_df[(TR_loc_df['move_in_date'] < find_date) == True]
    TRandDate_loc_df = TRandDate_loc_df[(TRandDate_loc_df['move_out_date'] > find_date) == True]
    TRandDate_loc_df.drop_duplicates('Person Reference', keep='first', inplace=True) # Drop any potential duplicated Person Refs on the same date
    TRandDate_loc_df.reset_index(drop=True,inplace=True)

    n_people = len(TRandDate_loc_df.index)
    n_children, n_adults, n_OAPs = 0, 0, 0
    age_adults = []

    for j, row_TRandDate in TRandDate_loc_df.iterrows():  # Iterate through people in tenancy at date of arrears instance
        
        age = (find_date - row_TRandDate['DATE OF BIRTH']).days / 365.25
        if age < 18.0:
            n_children = n_children+1
        else:
            n_adults = n_adults+1
            age_adults.append(age)
            if age > OAP_age_boundary:
                n_OAPs = n_OAPs+1
        if row_TRandDate['tin_main_tenant_ind'] == 'Y':
            age_main_ten = age
    
    if n_people==0: 
        n_empties = n_empties+1
        print('''Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = %f, Arrears date = %s''' %(find_TR,str(find_date)))
    
        Arrears_empty_tenancies = Arrears_empty_tenancies.append( Arrears_Events.loc[[i],[*Arrears_Events.columns.values[:8]]] )
        Arrears_empty_tenancies.loc[[i],['num_people']] = n_people
        
    else:
        # Fill Arrears_Events with calculated values
        Arrears_Events.loc[[i],['num_children','num_adults','num_OAPs','avg_adult_age','main_ten_age']] = n_children, n_adults, n_OAPs, np.mean(age_adults), age_main_ten

        # Aggregate categorised person features and add to Arrears_Events
        agg_rows = TRandDate_loc_df.groupby(['tcy_refno'], as_index=False).aggregate(sum)
        Arrears_Events.loc[[i],[*Arrears_Events.columns.values[14:]]] = agg_rows[agg_rows.columns.values[3:]].values.tolist()

Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 23391.000000, Arrears date = 2014-09-30 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 23571.000000, Arrears date = 2017-01-31 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 23633.000000, Arrears date = 2011-03-31 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 23633.000000, Arrears date = 2013-06-30 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 23640.000000, Arrears date = 2017-09-30 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 23641.000000, Arrears date = 2011-01-31 00:00:0

Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 23787.000000, Arrears date = 2011-04-30 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 23787.000000, Arrears date = 2013-10-31 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 23787.000000, Arrears date = 2016-08-31 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 23827.000000, Arrears date = 2011-11-30 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 23827.000000, Arrears date = 2015-05-31 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 23827.000000, Arrears date = 2017-10-31 00:00:0

Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 23904.000000, Arrears date = 2011-09-30 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 23904.000000, Arrears date = 2012-10-31 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 23904.000000, Arrears date = 2013-05-31 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 23904.000000, Arrears date = 2015-03-31 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 23907.000000, Arrears date = 2011-11-30 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 23907.000000, Arrears date = 2012-07-31 00:00:0

Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 23974.000000, Arrears date = 2016-07-31 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 23976.000000, Arrears date = 2011-05-31 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 23976.000000, Arrears date = 2013-10-31 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 23978.000000, Arrears date = 2012-11-30 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 23978.000000, Arrears date = 2013-10-31 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 23978.000000, Arrears date = 2014-10-31 00:00:0

Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 24024.000000, Arrears date = 2016-03-31 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 24024.000000, Arrears date = 2016-12-31 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 24024.000000, Arrears date = 2017-02-28 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 24024.000000, Arrears date = 2017-06-30 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 24024.000000, Arrears date = 2017-09-30 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 24024.000000, Arrears date = 2018-01-31 00:00:0

...

Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 55684.000000, Arrears date = 2011-01-31 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 55684.000000, Arrears date = 2011-10-31 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 55684.000000, Arrears date = 2012-08-31 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 55684.000000, Arrears date = 2012-10-31 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 55684.000000, Arrears date = 2013-02-28 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 55684.000000, Arrears date = 2013-04-30 00:00:0

Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 55760.000000, Arrears date = 2011-01-31 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 55760.000000, Arrears date = 2012-10-31 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 55760.000000, Arrears date = 2013-09-30 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 55760.000000, Arrears date = 2014-01-31 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 55760.000000, Arrears date = 2014-03-31 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 55760.000000, Arrears date = 2014-08-31 00:00:0

Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 55825.000000, Arrears date = 2011-01-31 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 55825.000000, Arrears date = 2017-06-30 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 55831.000000, Arrears date = 2011-01-31 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 55831.000000, Arrears date = 2012-01-31 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 55831.000000, Arrears date = 2013-04-30 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 55831.000000, Arrears date = 2016-06-30 00:00:0

Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 55903.000000, Arrears date = 2011-01-31 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 55903.000000, Arrears date = 2012-08-31 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 55903.000000, Arrears date = 2015-06-30 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 55903.000000, Arrears date = 2015-11-30 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 55903.000000, Arrears date = 2016-11-30 00:00:00
Warning - Arrears date does not coincide with dates of any people on the tenancy.
                          Ten Ref = 55903.000000, Arrears date = 2017-04-30 00:00:0

KeyboardInterrupt: 

In [66]:
Arrears_Events.head(10)

,ACCOUNT NUMBER,ConsecMonths,Current Tenancy,DatesArrears,LongTermIndicator,LongtermArrearsNum,MonthsInArrears,Tenancy Ref,Tenancy Type,num_children,num_adults,num_OAPs,avg_adult_age,main_ten_age,Lang_writ_ENGLISH,Lang_writ_not_ENGLISH,Lang_writ_nan,Gender_M,Gender_R,Gender_nan,Nationality_BRITISH,Nationality_not_BRITISH,Nationality_nan,disability_ND,disability_Y,Marital_stat_MAR,Marital_stat_SING,Marital_stat_not_married_not_single,Marital_stat_nan,Ethnicity_1,Ethnicity_12,Ethnicity_13,Ethnicity_other,Ethnicity_nan,At_risk_Y,At_risk_nan,Sexual_orientation_BI,Sexual_orientation_F,Sexual_orientation_GAY,Sexual_orientation_GL,Sexual_orientation_HET,Sexual_orientation_LES,Sexual_orientation_OTH,Sexual_orientation_REFD,Sexual_orientation_TRAN,Sexual_orientation_not_HET,Sexual_orientation_nan
0,9022637,4,1,2011-01-31,0,0,0,23372,monthly,0.0,2.0,0.0,52.907598,55.534565,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,9022637,20,1,2011-10-31,1,0,4,23372,monthly,0.0,2.0,0.0,53.655031,56.281999,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,9022637,12,1,2016-08-31,1,1,24,23372,monthly,0.0,2.0,0.0,58.490075,61.117043,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,9022646,1,1,2011-01-31,0,0,0,23381,monthly,0.0,2.0,0.0,37.801506,36.618754,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,9022646,22,1,2011-04-30,1,0,1,23381,monthly,0.0,2.0,0.0,38.045175,36.862423,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
5,9022647,24,1,2011-01-31,1,0,0,23382,monthly,0.0,1.0,0.0,40.416153,40.416153,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,9022647,11,1,2013-02-28,1,1,24,23382,monthly,0.0,1.0,0.0,42.494182,42.494182,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,9022647,11,1,2014-02-28,1,2,35,23382,monthly,0.0,1.0,0.0,43.493498,43.493498,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8,9022647,11,1,2015-02-28,1,3,46,23382,monthly,0.0,1.0,0.0,44.492813,44.492813,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9,9022647,11,1,2016-02-29,1,4,57,23382,monthly,0.0,1.0,0.0,45.494867,45.494867,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## Export  Arrears_Events_PersonsInfo to csv file

In [61]:
Arrears_Events.to_csv('Arrears_Events_PersonsInfo.csv')
